# 🥒 Pickle Mistral and Upload to Modelbit

## 💽 Install Prerequisites
Install necessary requirements before `pickle`. Make sure to restart your Colab instance/runtime after installation.

In [ ]:
!pip3 install modelbit protobuf==3.20 accelerate==0.25.0 bitsandbytes==0.41.3.post2 transformers==4.36.2 scipy==1.11.4 torch==2.1.0 cloudpickle==3.0.0

## 📤 Export Model
Load the model.

In [ ]:
import torch
import cloudpickle
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

device = "cuda"

# INT4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# INT8
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_quant_type="nf8",
#     bnb_8bit_use_double_quant=True,
# )

# Take out `load_in_4bit` and `quantization_config` for FP16.
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)


Feel free to rename the `.pkl` prior to running the export cell.

In [ ]:
with open('pipe-int4.pkl', 'wb') as file:
    cloudpickle.dump(pipe, file)

## 📦Upload to Modelbit
Authenticate to Modelbit.

In [ ]:
import modelbit as mb

mb.login()

Upload to Modelbit.

Note: This might take around ~10 minutes.

In [ ]:
mb.add_files("mistral_prompt", "pipe-int4.pkl")